In [29]:
def twoD_check(line_one, line_two):
    coord_one, delta_one = line_one
    x1, y1, z1 = coord_one
    dx1, dy1 = delta_one
    coord_two, delta_two = line_two
    x2, y2, z2 = coord_two
    dx2, dy2 = delta_two
    if dx1 == 0 or dx2 == 0:
        return False
    m_one = dy1 / dx1
    m_two = dy2 / dx2
    line_one = (m_one, y1 - m_one * x1)
    line_two = (m_two, y2 - m_two * x2)

    div = dx1 * dy2 - dy1 * dx2
    if div == 0:
        return False  
    d1 = x1 * (y1 + dy1) - y1 * (x1 + dx1)
    d2 = x2 * (y2 + dy2) - y2 * (x2 + dx2)
    x = -(d1 * dx2 - dx1 * d2) / div
    y = -(d1 * dy2 - dy1 * d2) / div
    if dx1 > 0 and x < x1:
        return False
    if dx2 > 0 and x < x2:
        return False
    if dx1 < 0 and x > x1:
        return False
    if dx2 < 0 and x > x2:
        return False
    if dy1 > 0 and y < y1:
        return False
    if dy2 > 0 and y < y2:
        return False
    if dy1 < 0 and y > y1:
        return False
    if dy2 < 0 and y > y2:
        return False
    return True

In [31]:
from sympy import Point, Line

text = open("input.txt").read().split("\n")
text = [[tuple(int(z) for z in y.split(", ")) for y in x.split(" @ ")] for x in text]

limit = 20

for vx in range(-limit, limit+1):
    print(vx)
    for vy in range(-limit, limit+1):
        if vx == 0 and vy == 0:
            continue
        two_lines = []
        for hailstone in text[:3]:
            position, velocity = hailstone
            moved_velocity = (velocity[0] - vx, velocity[1] - vy)
            two_lines.append((position, moved_velocity))
        i_one = twoD_check(two_lines[0], two_lines[1])
        i_two = twoD_check(two_lines[0], two_lines[2])
        if not i_one or not i_two:
            continue
        for vz in range(-limit, limit+1):
            if vx == 0 and vy == 0 and vz == 0:
                continue
            lines = []
            try:
                for hailstone in text[:3]:
                    position, velocity = hailstone
                    moved_velocity = (velocity[0] - vx, velocity[1] - vy, velocity[2] - vz)
                    lines.append(Line(Point(*position), Point(position[0] + moved_velocity[0], position[1] + moved_velocity[1], position[2] + moved_velocity[2])))
                intersect_one = lines[0].intersection(lines[1])
                intersect_two = lines[0].intersection(lines[2])
            except:
                continue
            if len(intersect_one) and len(intersect_two):
                print(intersect_one, intersect_two)
                break

-20
-19
-18
-17
-16
-15
-14
-13
-12
-11
-10
-9
-8
-7
[Point3D(36, 13, -38)] [Point3D(40, 13, -54)]
-6
[Point3D(33, 13, -26)] [Point3D(36, 13, -38)]
-5
[Point3D(30, 13, -14)] [Point3D(32, 13, -22)]
-4
[Point3D(27, 13, -2)] [Point3D(28, 13, -6)]
-3
[Point3D(24, 13, 10)] [Point3D(24, 13, 10)]
-2
-1
0
[Point3D(15, 13, 46)] [Point3D(12, 13, 58)]
1
[Point3D(12, 13, 58)] [Point3D(8, 13, 74)]
2
[Point3D(9, 13, 70)] [Point3D(4, 13, 90)]
3
4
5
6
7
8
9
10


In [6]:
from sympy import Symbol
from sympy import solve_poly_system

handle = open("input.txt","r")

shards = []
for line in handle:
  pos, vel = line.strip().split(" @ ")
  px,py,pz = pos.split(", ")
  vx,vy,vz = vel.split(", ")
  shards.append((int(px),int(py),int(pz),int(vx),int(vy),int(vz)))

#Part 2 uses SymPy. We set up a system of equations that describes the intersections, and solve it.
x = Symbol('x')
y = Symbol('y')
z = Symbol('z')
vx = Symbol('vx')
vy = Symbol('vy')
vz = Symbol('vz')

equations = []
t_syms = []
#the secret sauce is that once you have three shards to intersect, there's only one valid line
#so we don't have to set up a huge system of equations that would take forever to solve. Just pick the first three.
for idx,shard in enumerate(shards[:3]):
  #vx is the velocity of our throw, xv is the velocity of the shard we're trying to hit. Yes, this is a confusing naming convention.
  x0,y0,z0,xv,yv,zv = shard
  t = Symbol('t'+str(idx)) #remember that each intersection will have a different time, so it needs its own variable

  #(x + vx*t) is the x-coordinate of our throw, (x0 + xv*t) is the x-coordinate of the shard we're trying to hit.
  #set these equal, and subtract to get x + vx*t - x0 - xv*t = 0
  #similarly for y and z
  eqx = x + vx*t - x0 - xv*t
  eqy = y + vy*t - y0 - yv*t
  eqz = z + vz*t - z0 - zv*t

  equations.append(eqx)
  equations.append(eqy)
  equations.append(eqz)
  t_syms.append(t)

#To my great shame, I don't really know how this works under the hood.
result = solve_poly_system(equations,*([x,y,z,vx,vy,vz]+t_syms))
print(result[0][0] + result[0][1] + result[0][2]) #part 2 answer


786617045860267
